# Classic Design of Experiments II

In this notebook, we will learn about classic design of experiments (DoE) techniques.

We start with implementing a full factorial approach, learn about the Halton sequence, implement the Latin hypercube, and end with a comparison of space filling techniques.

### **Table of Contents**
1. [Full Factorial](#full-factorial)
2. [Halton Sequence](#haltonsequence)
3. [Latin Hypercube](#latin-hypercube)
4. [Comparison of Space Filling Techniques](#spacefilling-techniques)

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

### **1. Full Factorial** <a class="anchor" id="full-factorial"></a>

In this section, we implement the full factorial DoE technique. 

As a prerequisite, it is important to understand the general steps of designing an experiment, which are:

BEGIN SOLUTION

1. Define the objective of experiment - Clearly state the problem or goal you want to address with the experiment.
2. Identify the key factors (input variables) that might influence the response variable (output).
3. Select the appropriate design type based on your objectives and the number of factors to be investigated.
4. Determine the factor levels and range at which each factor will be set during the experiment. The factor levels should cover the practical range of operation.
5. Conduct the experiment, collect data, and analyze the results.

END SOLUTION
    

We implement the full factorial DoE technique [`full_fac`](../e2ml/experimentation/_full_factorial.py) in the [`e2ml.experimentation`](../e2ml/experimentation) subpackage. 

#### Question:
1. (a) What is the number of conditions for a full factorial design with levels [4, 5, 10] ? Verify it using your code.
    
    BEGIN SOLUTION

    The size of the full factorial design with $k$ factors $x_{1}, ....,x_{k}$, having $L_{1}, ..., L_{k}$ levels, is $N = \prod_{i=1}^{k}L_{i}$. So for this example, we have $k=3$ factors with levels $L_1=4, L_2=5, L_3=10$, resulting in the size of $4 \cdot 5 \cdot 10 = 200$. The shape of the matrix will be $(200, 3)$ with $N=200$ conditions for $k=3$ factors.

    END SOLUTION

In [ ]:
from e2ml.experimentation import full_fac

# BEGIN SOLUTION
levels = [4, 5, 10]
# Check the shape of the DoE matrix `X` for the given levels.
X = full_fac(levels) 
print(X.shape)
# END SOLUTION

#### Question:
1. (b) What is the major disadvantage of the full factorial technique?

    BEGIN SOLUTION

    The size of the full factorial design grows very quickly with increasing numbers of levels and factors. Thus, it becomes computationally expensive for higher number of levels and factors.


    END SOLUTION

### **2. Halton Sequence** <a class="anchor" id="haltonsequence"></a>


Now, we will implement the Halton sequence as a **space filling** technique for experimental design. 
We will use this implementation and compare it to Lating hypercube and uniform sampling as two other space filling DOE techniques.

A Halton sequence is a low-discrepancy sequence with the property that for all values of $N$, its subsequence $\mathbf{x}_1, ..., \mathbf{x}_N$ has a low **discrepancy**. Roughly said, a low discrepancy is a kind of criterion to express the fact that a sequence fills a segment, leaving no gaps. Low-discrepancy sequences are also called quasirandom sequences, due to their common use as a replacement of uniformly distributed random numbers. The *quasi* modifier is used to denote more clearly that the values of a low-discrepancy sequence are neither random nor pseudorandom. Still, such sequences share some properties of random variables.

Quasirandom numbers have an advantage over pure random numbers in that they cover the domain of interest quickly and evenly. They have an advantage over purely deterministic methods because they only give high accuracy when the number of conditions is preset. In contrast, using quasirandom sequences improves the accuracy as more conditions are added, with full reuse of the existing points. On the other hand, quasirandom point sets can have a significantly lower discrepancy for a given number of points than purely random sequences.

Halton sequences are constructed from one-dimensional van der Corput sequences. A van der Corput sequence is constructed by reversing the base-$n$ representation of the sequence of natural numbers, i.e., $1, 2, 3, \dots$.

For a given base $b = 2, 3, \dots$, the $b$-adic expansion of the positive integer $n \geq 1$ is expressed as

\begin{equation*}
    n = \sum_{j=1}^{T} a_j b^{j-1},
\end{equation*}

where $0 \leq a_j < b$ are the integer coefficients of the expansion. Accordingly, the $n$-th number in the van der Corput sequence is defined through

\begin{equation*}
    \phi_b(n) = \sum_{j=1}^{T} \frac{a_j}{b^{j}}.
\end{equation*}

#### Question:
2. (a) Which numbers are represented through $\phi_2(7)$ and $\phi_3(5)$? Use the above equations to answer this question.
   
   The $b=2$-adic expansion of $n=7$ is given through
   \begin{equation*}
       7 = 1 \cdot 2^0 + 1 \cdot 2^1  + 1 \cdot 2^2,
   \end{equation*}
   such that $a_1 = a_2 = a_3 = 1$. Accordingly, we obtain
   \begin{equation*}
       \phi_2(7) = \frac{1}{2^1} + \frac{1}{2^2} + \frac{1}{2^3} = \frac{1}{2} + \frac{1}{4} + \frac{1}{8} = \frac{7}{8} = 0.875.
   \end{equation*}
   
   The $b=3$-adic expansion of $n=5$ is given through: 
   \begin{equation*}
       5 = 2 \cdot 3^0 + 1 \cdot 3^1,
   \end{equation*}
   such that $a_1 = 2, a_2 = 1$. Accordingly, we obtain:
   \begin{equation*}
       \phi_3(5) = \frac{2}{3^1} + \frac{1}{3^2} = \frac{6}{9} + \frac{1}{9} = \frac{7}{9} \approx 0.778.
   \end{equation*}
   
   
With this knowledge, we implement the function [`van_der_corput_sequence`](../e2ml/experimentation/_halton.py) in the [`e2ml.experimentation`](../e2ml/experimentation) subpackage.
Once, the implementation has been completed, we check its validity for $b=10$ and $n_\mathrm{max}=20$.

In [ ]:
from e2ml.experimentation import van_der_corput_sequence
exp_sequence = np.array([
    0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.01, 0.11, 0.21, 0.31, 0.41, 0.51, 0.61, 0.71, 0.81, 0.91, 0.02
])
np.testing.assert_array_almost_equal(van_der_corput_sequence(20, 10), exp_sequence)

Multi-dimensional Halton sequences are constructed by van der Corput sequences that use so-called [coprime numbers](https://en.wikipedia.org/wiki/Coprime_integers) as its bases. For this purpose, we implement the function [`primes_from_2_to`](../e2ml/experimentation/_halton.py) in the [`e2ml.experimentation`](../e2ml/experimentation) subpackage. This function generates prime numbers from $2$ to $n_\mathrm{max}$. We check the function's validity for the prime numbers until $50$.

In [ ]:
from e2ml.experimentation import primes_from_2_to
exp_prime_numbers = np.array([2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47])
np.testing.assert_array_equal(primes_from_2_to(50), exp_prime_numbers)

Given the van der Corput sequence and primer numbers generator, we can now implement a multi-dimensional Halton sequence by using for each dimension one van der Corput sequence with a unique prime number as basis, i.e., each dimension must use a different prime number.
We implement the function [`halton_unit`](../e2ml/experimentation/_halton.py) in the [`e2ml.experimentation`](../e2ml/experimentation) subpackage. It generates conditions $\mathbf{x} \in [0, 1]^D$.

In [ ]:
from e2ml.experimentation import halton_unit
# Generate 200 two-dimensional conditions as `X` with the Halton sequence.
X = halton_unit(200, 2) # <-- SOLUTION

# Plot generated conditions.
plt.scatter(X[:, 0], X[:, 1])
plt.xlabel('$x_1$', fontsize=15)
plt.ylabel('$x_2$', fontsize=15)
plt.show()

In many experiments, we want to generate conditions whose levels lie in specific intervals for each dimension.

#### Question:
2. (b) How to scale the values $x_1, \dots, x_N \in [0, 1]$ such that all of them lie in the interval $[a, b]$ with $a,b \in \mathbb{R}$.

   BEGIN SOLUTION
   
   The transformation is defined as:
   \begin{equation*}
       z_n = x_n \cdot (b - a) + a.
   \end{equation*}
   END SOLUTION
   
We extend the function [`halton_unit`](../e2ml/experimentation/_halton.py) by allowing to define boundaries for each dimension's interval through scaling. Therefore, we implement the function [`halton`](../e2ml/experimentation/_halton.py) in the [`e2ml.experimentation`](../e2ml/experimentation) subpackage.

In [ ]:
from e2ml.experimentation import halton

# Generate 200 two-dimensional conditions with the Halton sequence
# in the hypercube [-2, 2] x [1, 5].
bounds = np.array([[-2, 2], [1, 5]]) 
X = halton(200, 2, bounds) 

# Plot generated conditions.
plt.scatter(X[:, 0], X[:, 1])
plt.xlabel('$x_1$', fontsize=15)
plt.ylabel('$x_2$', fontsize=15)
plt.show()

### **3. Latin Hypercube** <a class="anchor" id="latin-hypercube"></a>

Now, we will implement the Latin hypercube technique for experimental design.
We implement the function [`lat_hyp_cube_unit`](../e2ml/experimentation/_latin_hypercube.py) in the [`e2ml.experimentation`](../e2ml/experimentation) subpackage. It generates conditions $\mathbf{x} \in [0, 1]^D$.

In [ ]:
from e2ml.experimentation import lat_hyp_cube_unit

# Generate 200 two-dimensional conditions `X` with the Latin hypercube.
X = lat_hyp_cube_unit(200, 2) # <-- SOLUTION

# Plot generated conditions.
plt.scatter(X[:, 0], X[:, 1])
plt.xlabel('$x_1$', fontsize=15)
plt.ylabel('$x_2$', fontsize=15)
plt.show()

Similar to Halton, we extend the function [`lat_hyp_cube_unit`](../e2ml/experimentation/_latin_hypercube.py) by allowing to define boundaries for each dimension's interval through scaling. Therefore, we implement the function [`lat_hyp_cube`](../e2ml/experimentation/_latin_hypercube.py) in the [`e2ml.experimentation`](../e2ml/experimentation) subpackage.

In [ ]:
from e2ml.experimentation import lat_hyp_cube

# Generate 200 two-dimensional conditions `X` with the Latin hypercube
# in the hypercube [-2, 2] x [1, 5].
bounds = np.array([[-2, 2], [1, 5]]) 
X = lat_hyp_cube(200, 2, bounds) 

# Plot generated conditions.
plt.scatter(X[:, 0], X[:, 1])
plt.xlabel('$x_1$', fontsize=15)
plt.ylabel('$x_2$', fontsize=15)
plt.show()

### **4. Comparison of Space Filling Techniques** <a class="anchor" id="spacefilling-techniques"></a>


In this section, we compare the Halton sequence, Latin hypercube, and uniform sampling. First, we compare them just visually. Therefore, we plot 200 conditions generated in the two-dimensional unit cube by the respective technique.

In [ ]:
# Visual comparison of Halton sequence, Latin hypercube, and uniform sampling.
# BEGIN SOLUTION
X_halton = halton(200, 2) 
X_latin = lat_hyp_cube(200, 2)
X_random = np.random.uniform(size=(200, 2))
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))
ax[0].set_title('Halton sequence', fontsize=15)
ax[0].scatter(X_halton[:, 0], X_halton[:, 1])
ax[0].set_xlabel('$x_1$', fontsize=15)
ax[0].set_ylabel('$x_2$', fontsize=15)
ax[1].set_title('Latin hypercube', fontsize=15)
ax[1].scatter(X_latin[:, 0], X_latin[:, 1])
ax[1].set_xlabel('$x_1$', fontsize=15)
ax[1].set_ylabel('$x_2$', fontsize=15)
ax[2].set_title('Uniform sampling', fontsize=15)
ax[2].scatter(X_random[:, 0], X_random[:, 1])
ax[2].set_xlabel('$x_1$', fontsize=15)
ax[2].set_ylabel('$x_2$', fontsize=15)
plt.show()
# END SOLUTION

For a quantitative comparison, we want to approximate the integral of a cosine-shaped function in the interval $[-\pi, +\pi]$:

\begin{align*}
    f(x) &= \cos(x) + 1, \\
    F(x) &= \sin(x) + x + C.
\end{align*}

Accordingly, the definite integral in the interval $[-\pi, +\pi]$ is given through:

BEGIN SOLUTION

\begin{equation*}
    F(\pi) - F(-\pi) = 2\pi.
\end{equation*}

END SOLUTION

#### Question:
4. (a) Imagine, we can only generate conditions of the form $(x_1, x_2)^\mathrm{T}$ with $x_1 \in [-\pi, +\pi], x_2 \in [0, 2]$ and can only ask for the information whether $x_2 \in [0, f(x_1)]$ or $x_2 \in (f(x_1), 2]$ is true. How can we approximate the above definite integral through sampling and making use of this information?

   BEGIN SOLUTION
   
   We can generate conditions in the space $[-\pi, \pi] \times [0, 2]$, which has the area $4\pi$ and compute the proportion $\rho \in [0, 1]$ of conditions fulfilling $x_2 \in (f(x_1), 2]$. Accordingly, the approximated area is $\rho \cdot 4\pi$.
   
   END SOLUTION
   
In the following, we compare the error of uniform sampling, Halton sequence, and Latin hypercube to approximate the above definite integral for different numbers of conditions.

In [ ]:
def compute_approximation_error(X):
    """
    Computes mean squared error for the definite integral of
    function f(x) = cos(x) + 1 in the interval [-pi, pi].
    
    Parameters
    ----------
    X : numpy.ndarray of shape (n_conditions, 2)
        Conditions with `X[:, 0]` in [-pi, pi] and `X[:, 1]` in [0, 2].
        
    Returns
    -------
    error : float
        Mean squared error between approximated and true value of
        the definite integral.
    """
    # BEGIN SOLUTION
    f = np.cos(X[:, 0]) + 1
    rho = np.mean(X[:, 1] <= f) 
    error = (rho * 4 * np.pi - 2 * np.pi)**2
    return error
    # END SOLUTION

# Numbers of conditions to be tested.
n_conditions_list = np.linspace(10, 200, 10, dtype=int)

# Compute approximation errors for Halton sequence.
# BEGIN SOLUTION
errors_halton = np.zeros_like(n_conditions_list, dtype=float)
for i, n_conditions in enumerate(n_conditions_list):
    X_halton = halton(int(n_conditions), 2, bounds=[[-np.pi, np.pi], [0, 2]])
    errors_halton[i] = compute_approximation_error(X_halton)
# END SOLUTION

# Compute approximation errors for uniform sampling.
# BEGIN SOLUTION
n_trials = 100
errors_random = np.zeros((len(n_conditions_list), n_trials), dtype=float)
for j in range(n_trials):
    for i, n_conditions in enumerate(n_conditions_list):
        X_random = np.random.uniform(size=(n_conditions, 2))
        X_random[:, 0] = X_random[:, 0] * (np.pi - (-np.pi)) - np.pi
        X_random[:, 1] = X_random[:, 1] * (2 - 0) - 0
        errors_random[i, j] = compute_approximation_error(X_random)
errors_random_mean = errors_random.mean(axis=1)
# END SOLUTION

# Compute approximation errors for Latin Hypercube.
# BEGIN SOLUTION
n_trials = 100
errors_latin = np.zeros((len(n_conditions_list), n_trials), dtype=float)
for j in range(n_trials):
    for i, n_conditions in enumerate(n_conditions_list):
        X_latin = lat_hyp_cube(int(n_conditions), 2, bounds=[[-np.pi, np.pi], [0, 2]])
        errors_latin[i, j] = compute_approximation_error(X_latin)
errors_latin_mean = errors_latin.mean(axis=1)
# END SOLUTION

# Plot computed approximation errors along numbers of generated conditions.
# BEGIN SOLUTION
plt.figure()    
plt.plot(n_conditions_list, errors_halton, label='Halton sequence')
plt.plot(n_conditions_list, errors_random_mean, label='Uniform sampling')
plt.plot(n_conditions_list, errors_latin_mean, label='Latin hypercube')
plt.xlabel('# conditions', fontsize=15)
plt.ylabel('approximation error', fontsize=15)
plt.legend(prop={'size': 15})
plt.show()
# END SOLUTION